In [3]:
from constraint import Problem, AllDifferentConstraint

def solve_sudoku(grid):
    cells = [(r, c) for r in range(9) for c in range(9)]
    
    problem = Problem()
    problem.addVariables(cells, range(1, 10))
    
    for r in range(9):
        problem.addConstraint(AllDifferentConstraint(),
                              [(r, c) for c in range(9)])

    for c in range(9):
        problem.addConstraint(AllDifferentConstraint(),
                              [(r, c) for r in range(9)])

        for br in range(3):
            for bc in range(3):
                block = [
                    (r, c)
                    for r in range(br*3, br*3 + 3)
                    for c in range(bc*3, bc*3 + 3)
                ]
                problem.addConstraint(AllDifferentConstraint(), block)
    
    for r in range(9):
        for c in range(9):
            if grid[r][c] != 0:
                # unary constraint X[r,c] == grid[r][c]
                val = grid[r][c]
                problem.addConstraint(lambda x, v=val: x == v, [(r, c)])
    
    solution = problem.getSolution()
    if not solution:
        print("No solution found.")
        return None
    
    solved = [[solution[(r, c)] for c in range(9)] for r in range(9)]
    return solved

if __name__ == "__main__":

    grid = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9],
    ]
    
    solution = solve_sudoku(grid)
    if solution:
        for row in solution:
            print(row)


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
